In [121]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

### PART 2

#### -------- Load and prepare the dataset --------

In [122]:
# Load the dataset
day_df = pd.read_csv('data/day.csv')
hour_df = pd.read_csv('data/hour.csv')

# Add source column to each dataset
# day_df['source'] = 'day'
# hour_df['source'] = 'hour'

# Merge the datasets
merged_df = pd.concat([day_df, hour_df], ignore_index=True)
merged_df = merged_df.drop(['dteday'], axis=1)

#TEST İÇİN 10 VERİYE İN
merged_df = merged_df.head(10)

X = merged_df.drop('cnt', axis=1).values
y = merged_df['cnt'].values
    
X.shape, y.shape


((10, 15), (10,))

In [123]:
merged_df

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hr
0,1,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,NaN
1,2,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,NaN
2,3,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,NaN
3,4,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,NaN
4,5,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,NaN
5,6,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,88,1518,1606,NaN
6,7,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510,NaN
7,8,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,891,959,NaN
8,9,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,54,768,822,NaN
9,10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321,NaN


#### -------- Develop custom distance functions ---------

In [124]:
def manhattan_distance(x1, x2):
    return np.sum(np.abs(x1 - x2))


#### -------- KNN classifier --------

In [125]:
def knn_predict_regressor(X_train, y_train, X_test, k):
    print('KNN Regressor Start')

    y_pred = []
    for test_point in X_test:
        distances = []

        # Manhattan a göre hesapla
        for i in range(len(X_train)):
            dist = manhattan_distance(test_point, X_train[i])
            distances.append((dist, y_train[i]))
        
        distances.sort(key=lambda x: x[0])
        k_nearest = distances[:k]
        k_nearest_values = [pair[1] for pair in k_nearest]
        
        prediction = (sum(k_nearest_values) / len(k_nearest_values)) 
        
        y_pred.append(prediction)

    print('KNN Regressor End')
    
    return np.array(y_pred)



#### k-fold cross-validation

In [ ]:
def k_fold_cross_validation(X, y, k_neighbors, n_folds=3):
    print('K-Fold Cross Validation Start')

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    conf_matrices = []
    run_times = []
    fold_results = []

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    # verileri scale et. bazı değerler küçük bazı büyük. standardize et. 
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    fold_num = 1
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        print('y_test:', y_test)
        # y'yi float yap. predict float oldugundan accuracy hesaplanırken hata veriyor.
        y_test = y_test.astype(float)
        print('y_test float:', y_test)
        
        # KNN Çalıştır 
        start_time = time.time()
        y_pred = knn_predict_regressor(X_train, y_train, X_test, k_neighbors)
        end_time = time.time()
        run_time = end_time - start_time
        
        # Performans hesapla
        print('y_test:', y_test)
        print('y_pred:', y_pred)
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        # performansları listelerine ekle
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        conf_matrices.append(conf_matrix)
        run_times.append(run_time)
        
        # tümünü fold result listesinde topla
        fold_results.append({
            'fold': fold_num,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'conf_matrix': conf_matrix,
            'run_time': run_time
        })
        
        fold_num += 1

    print('K-Fold Cross Validation End')
    
    return {
        'accuracy': np.mean(accuracy_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'f1': np.mean(f1_scores),
        'conf_matrices': conf_matrices,
        'run_time': np.mean(run_times),
        'fold_results': fold_results
    }


#### Create a function to plot confusion matrix

In [127]:
def plot_confusion_matrix(conf_matrices, title):
    avg_conf_matrix = np.mean(conf_matrices, axis=0)
    plt.figure(figsize=(8, 6))
    
    sns.heatmap(avg_conf_matrix, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=['True (0)', 'False (1)'],
                yticklabels=['True (0)', 'False (1)'])
    
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix: {title}')
    # plt.tight_layout()
    plt.show()


#### Run all functions

In [128]:

print("KNN Code starts here")
k = 3

# Çalıştır 
overall_start_time = time.time()
results = k_fold_cross_validation(X, y, k)
overall_end_time = time.time()
overall_run_time = overall_end_time - overall_start_time

print("KNN Code ok. ")


KNN Code starts here
K-Fold Cross Validation Start
KNN Regressor Start
KNN Regressor End
y_test: [ 985  801 1606  822]
y_pred: [1503.66666667 1503.66666667 1503.66666667 1503.66666667]


/Users/fatih/work/prj/GTU/ML/HW1/ML_WORK1/.conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/fatih/work/prj/GTU/ML/HW1/ML_WORK1/.conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/fatih/work/prj/GTU/ML/HW1/ML_WORK1/.conda/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

#### Sonuçlar;

In [ ]:
# Fold bazında sonuçları yaz
print("\n --- Results by Fold:")
print("-" * 100)
print(f"{'Fold':<6} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1 Score':<10} {'Run Time':<10}")
print("-" * 100)

for fold_result in results['fold_results']:
    print(f"{fold_result['fold']:<6} {fold_result['accuracy']:.4f} {fold_result['precision']:.4f} {fold_result['recall']:.4f} {fold_result['f1']:.4f} {fold_result['run_time']:.4f}s")

# Modelin sonucu
print("\n--- Results Summary:")
print("-" * 50)
print(f"Configuration: k={k}")
print(f"Average Accuracy: {results['accuracy']:.4f}")
print(f"Average Precision: {results['precision']:.4f}")
print(f"Average Recall: {results['recall']:.4f}")
print(f"Average F1 Score: {results['f1']:.4f}")
print(f"Average Fold Run Time: {results['run_time']:.4f}s")
print(f"Total Run Time: {overall_run_time:.4f}s")

# Grafiği çiz.
plot_confusion_matrix(results['conf_matrices'], f"k={k}")


 --- Results by Fold:
----------------------------------------------------------------------------------------------------
Fold   Accuracy   Precision  Recall     F1 Score   Run Time  
----------------------------------------------------------------------------------------------------
1      0.0000 0.0000 0.0000 0.0000 0.0002s
2      0.0000 0.0000 0.0000 0.0000 0.0002s
3      0.0000 0.0000 0.0000 0.0000 0.0003s

--- Results Summary:
--------------------------------------------------
Configuration: k=3
Average Accuracy: 0.0000
Average Precision: 0.0000
Average Recall: 0.0000
Average F1 Score: 0.0000
Average Fold Run Time: 0.0002s
Total Run Time: 0.0288s


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.